In [1]:
import pandas as pd
import numpy as np

In [29]:
file_path3= "C:/Users/vgarciario/desktop/UA MASTER/labo3/productos_a_predecir.txt"
productos_predecir = pd.read_csv(file_path3, sep='\t')

file_path = 'C:/Users/vgarciario/desktop/UA MASTER/labo3/sell-in.txt'
sell_in = pd.read_csv(file_path, sep='\t')

In [30]:
# Definir los periodos de interés
#periodos = [201701,201702,201703,201704,201705,201706,201707,201708,201709,201710,201711,201712,201801,201802,201803,201804,201805,201806,201807,201808,201809,201810,201811,201812,201901,201902,201903,201904,201905,201906,201907,201908,201909,201910,201911,201912]
periodos = [201701,201702,201703,201704,201705,201706,201707,201708,201709,201710,201711,201712,201801,201802,201803,201804,201805,201806,201807,201808,201809,201810,201811,201812,201901,201902,201903,201904,201905,201906,201907,201908,201909,201910,201911,201912]
# Filtrar las filas correspondientes a los periodos de interés
sell_in_completo = sell_in[sell_in['periodo'].isin(periodos)]

In [31]:
dataset = pd.merge(productos_predecir, sell_in_completo, on='product_id', how='inner')
sell_in_completo = dataset.groupby(['periodo', 'product_id'])['tn'].sum()#.reset_index()
#sell_in_completo['periodo'] = pd.to_datetime(sell_in_completo['periodo'].astype(str) + '01', format='%Y%m%d')
#sell_in_completo.set_index("periodo", inplace=True)

In [28]:
sell_in_completo.head

<bound method NDFrame.head of             product_id          tn
periodo                           
2017-01-01       20001   934.77222
2017-01-01       20002   550.15707
2017-01-01       20003  1063.45835
2017-01-01       20004   555.91614
2017-01-01       20005   494.27011
...                ...         ...
2019-12-01       21263     0.01270
2019-12-01       21265     0.05007
2019-12-01       21266     0.05121
2019-12-01       21267     0.01569
2019-12-01       21276     0.00892

[22349 rows x 2 columns]>

In [9]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Dropout, Activation
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping


# Lista para acumular las filas de resultados
resultados_por_producto = []

# Elegir un número de pasos de tiempo
n_steps = 18  # Ventana de tiempo de 12 meses
n_features = 1  # Cambia esto si tienes más características

# Inicializar scaler
scaler = MinMaxScaler()

# Preparar datos para clustering usando únicamente la característica 'tn'
sell_in_completo['periodo'] = pd.to_datetime(sell_in_completo['periodo'])
sell_in_completo = sell_in_completo.sort_values(by='periodo')
tn_data = sell_in_completo.pivot(index='periodo', columns='product_id', values='tn').fillna(0)
tn_data_normalizada = scaler.fit_transform(tn_data)

# Aplicar K-means clustering
num_clusters = 6  # Número de clusters
kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(tn_data_normalizada.T)
sell_in_completo['cluster'] = sell_in_completo['product_id'].map(
    dict(zip(tn_data.columns, kmeans.labels_))
)

# Función para crear secuencias de tiempo
def crear_secuencias(datos, n_steps, step_ahead=1):
    X, y = [], []
    for i in range(len(datos)):
        end_ix = i + n_steps
        out_end_ix = end_ix + step_ahead - 1
        if out_end_ix > len(datos) - 1:
            break
        seq_x, seq_y = datos[i:end_ix], datos[out_end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

# Definir la función para construir el modelo
def build_model():
    model = Sequential()
    model.add(Bidirectional(LSTM(units=256, return_sequences=True), input_shape=(n_steps, n_features)))
    model.add(Dropout(0.3))
    model.add(Bidirectional(LSTM(units=128)))
    model.add(Dropout(0.3))
    model.add(Dense(64))
    model.add(Activation('tanh'))
    model.add(Dense(1))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss='mse')
    return model

# Inicializar EarlyStopping
early_stopping = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)

# Diccionario para realizar un seguimiento de los productos ya procesados
productos_procesados = {}

# Entrenar y predecir para cada cluster
for cluster in range(num_clusters):
    productos_cluster = sell_in_completo[sell_in_completo["cluster"] == cluster]["product_id"].unique()
    aux_combined = []
    
    for producto in productos_cluster:
        aux = sell_in_completo[sell_in_completo["product_id"] == producto].drop(columns=["product_id", "cluster", "periodo"]).values
        aux_normalizado = scaler.fit_transform(aux)
        aux_combined.append(aux_normalizado)

    aux_combined = np.concatenate(aux_combined, axis=0)
    X, y = crear_secuencias(aux_combined, n_steps, step_ahead=2)
    X = X.reshape((X.shape[0], X.shape[1], n_features))

    # Construir el modelo
    model = build_model()

    # Entrenar el modelo con todos los datos del cluster
    model.fit(X, y, epochs=200, batch_size=32, callbacks=[early_stopping], verbose=0)

    # Realizar predicciones individuales para cada producto del cluster
    for producto in productos_cluster:
        if producto not in productos_procesados:
            try:
                aux = sell_in_completo[sell_in_completo["product_id"] == producto].drop(columns=["product_id", "cluster", "periodo"]).values
                aux_normalizado = scaler.fit_transform(aux)
                ultima_secuencia = aux_normalizado[-n_steps:].reshape((1, n_steps, n_features))
                prediccion_normalizada = model.predict(ultima_secuencia)
                prediccion = scaler.inverse_transform(prediccion_normalizada)
                resultados_por_producto.append({'product_id': producto, 'prediccion_1': prediccion[0][0]})
                productos_procesados[producto] = True  # Marcar producto como procesado
            except Exception as e:
                print(f"Error al procesar el producto {producto}: {str(e)}")
                resultados_por_producto.append({'product_id': producto, 'prediccion_1': 0})
                productos_procesados[producto] = True  # Marcar producto como procesado

# Convertir la lista de resultados en un DataFrame
predicciones_por_producto = pd.DataFrame(resultados_por_producto)

# Mostrar el DataFrame con las predicciones por producto
print(predicciones_por_producto)

# Guardar el DataFrame con las predicciones por producto en un archivo CSV
predicciones_por_producto.to_csv("C:/Users/vgarciario/desktop/UA MASTER/labo3/lstmclus126.csv", index=False)


c:\Users\vgARCIARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 618ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━

c:\Users\vgARCIARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 619ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Error al procesar el producto 21233: cannot reshape array of size 16 into shape (1,18,1)
Error al procesar el producto 21209: cannot reshape array of size 16 into shape (1,18,1)
Error al procesar el producto 21191: cannot reshape array of size 16 into shape (1,18,1)
Error al procesar el producto 21245: cannot reshape array of size 16 into shape (1,18,1)
Error al procesar el producto 21226: cannot reshape array of size 16 into shape (1,18,1)
Error al procesar el producto 21142: cannot reshape array of size 16 into shape (1,18,1)
Error al procesar el producto 21263: cannot reshape array of size 15 into shape (1,18,1)
Error al procesar el producto 21262: cannot reshape array of size 15 into shape (1,18,1)
Error al procesar el producto 21244: cannot reshape array of size 15 into shape (1,18,1)
Error al procesar el producto 21246: cannot re

c:\Users\vgARCIARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 726ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━

c:\Users\vgARCIARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 586ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━

c:\Users\vgARCIARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 615ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Error al procesar el producto 20089: cannot reshape array of size 17 into shape (1,18,1)
Error al procesar el producto 20679: cannot reshape array of size 17 into 

In [12]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Dropout, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Lista para acumular las filas de resultados
resultados_por_producto = []

# Elegir un número de pasos de tiempo
n_steps = 12  # Ventana de tiempo de 12 meses
n_features = 1  # Cambia esto si tienes más características

# Inicializar scaler
scaler = MinMaxScaler()

# Preparar datos para clustering usando únicamente la característica 'tn'
sell_in_completo['periodo'] = pd.to_datetime(sell_in_completo['periodo'])
sell_in_completo = sell_in_completo.sort_values(by='periodo')
tn_data = sell_in_completo.pivot(index='periodo', columns='product_id', values='tn').fillna(0)
tn_data_normalizada = scaler.fit_transform(tn_data)

# Aplicar K-means clustering
num_clusters = 5  # Número de clusters, se puede experimentar con este valor
kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(tn_data_normalizada.T)
sell_in_completo['cluster'] = sell_in_completo['product_id'].map(
    dict(zip(tn_data.columns, kmeans.labels_))
)

# Función para crear secuencias de tiempo
def crear_secuencias(datos, n_steps, step_ahead=1):
    X, y = [], []
    for i in range(len(datos)):
        end_ix = i + n_steps
        out_end_ix = end_ix + step_ahead - 1
        if out_end_ix > len(datos) - 1:
            break
        seq_x, seq_y = datos[i:end_ix], datos[out_end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

# Definir la función para construir el modelo
def build_model():
    model = Sequential()
    model.add(Bidirectional(LSTM(units=256, return_sequences=True), input_shape=(n_steps, n_features)))
    model.add(Dropout(0.3))
    model.add(Bidirectional(LSTM(units=128)))
    model.add(Dropout(0.3))
    model.add(Dense(64))
    model.add(Activation('tanh'))
    model.add(Dense(1))
    model.compile(optimizer=Adam(learning_rate=1e-3), loss='mse')
    return model

# Inicializar EarlyStopping
early_stopping = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)

# Diccionario para realizar un seguimiento de los productos ya procesados
productos_procesados = {}

# Entrenar y predecir para cada cluster
for cluster in range(num_clusters):
    productos_cluster = sell_in_completo[sell_in_completo["cluster"] == cluster]["product_id"].unique()
    aux_combined = []
    
    for producto in productos_cluster:
        aux = sell_in_completo[sell_in_completo["product_id"] == producto].drop(columns=["product_id", "cluster", "periodo"]).values
        aux_normalizado = scaler.fit_transform(aux)
        aux_combined.append(aux_normalizado)

    aux_combined = np.concatenate(aux_combined, axis=0)
    X, y = crear_secuencias(aux_combined, n_steps, step_ahead=2)
    X = X.reshape((X.shape[0], X.shape[1], n_features))

    # Construir el modelo
    model = build_model()

    # Entrenar el modelo con todos los datos del cluster
    model.fit(X, y, epochs=200, batch_size=32, callbacks=[early_stopping], verbose=0)

    # Realizar predicciones individuales para cada producto del cluster
    for producto in productos_cluster:
        if producto not in productos_procesados:
            try:
                aux = sell_in_completo[sell_in_completo["product_id"] == producto].drop(columns=["product_id", "cluster", "periodo"]).values
                aux_normalizado = scaler.fit_transform(aux)
                ultima_secuencia = aux_normalizado[-n_steps:].reshape((1, n_steps, n_features))
                prediccion_normalizada = model.predict(ultima_secuencia)
                prediccion = scaler.inverse_transform(prediccion_normalizada)
                resultados_por_producto.append({'product_id': producto, 'prediccion_1': prediccion[0][0]})
                productos_procesados[producto] = True  # Marcar producto como procesado
            except Exception as e:
                print(f"Error al procesar el producto {producto}: {str(e)}")
                resultados_por_producto.append({'product_id': producto, 'prediccion_1': 0})
                productos_procesados[producto] = True  # Marcar producto como procesado

# Convertir la lista de resultados en un DataFrame
predicciones_por_producto = pd.DataFrame(resultados_por_producto)

# Mostrar el DataFrame con las predicciones por producto
print(predicciones_por_producto)

# Guardar el DataFrame con las predicciones por producto en un archivo CSV
predicciones_por_producto.to_csv("C:/Users/vgarciario/desktop/UA MASTER/labo3/lstmclus2.csv", index=False)


c:\Users\vgARCIARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 670ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━

c:\Users\vgARCIARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 562ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Error al procesar el producto 20032: cannot reshape array of size 11 into shape (1,12,1)
Error al procesar el producto 21153: cannot reshape array of size 11 into shape (1,12,1)
Error al procesar el producto 21159: cannot reshape array of 

c:\Users\vgARCIARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 818ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━

c:\Users\vgARCIARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 819ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━

c:\Users\vgARCIARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 563ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━

In [13]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Dropout, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Lista para acumular las filas de resultados
resultados_por_producto = []

# Elegir un número de pasos de tiempo
n_steps = 12  # Ventana de tiempo de 12 meses
n_features = 1  # Cambia esto si tienes más características

# Inicializar scaler
scaler = MinMaxScaler()

# Preparar datos para clustering usando únicamente la característica 'tn'
sell_in_completo['periodo'] = pd.to_datetime(sell_in_completo['periodo'])
sell_in_completo = sell_in_completo.sort_values(by='periodo')
tn_data = sell_in_completo.pivot(index='periodo', columns='product_id', values='tn').fillna(0)
tn_data_normalizada = scaler.fit_transform(tn_data)

# Aplicar K-means clustering
num_clusters = 5  # Número de clusters, se puede experimentar con este valor
kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(tn_data_normalizada.T)
sell_in_completo['cluster'] = sell_in_completo['product_id'].map(
    dict(zip(tn_data.columns, kmeans.labels_))
)

# Función para crear secuencias de tiempo
def crear_secuencias(datos, n_steps, step_ahead=1):
    X, y = [], []
    for i in range(len(datos)):
        end_ix = i + n_steps
        out_end_ix = end_ix + step_ahead - 1
        if out_end_ix > len(datos) - 1:
            break
        seq_x, seq_y = datos[i:end_ix], datos[out_end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

# Definir la función para construir el modelo
def build_model():
    model = Sequential()
    model.add(Bidirectional(LSTM(units=128, return_sequences=True), input_shape=(n_steps, n_features)))
    model.add(Dropout(0.2))
    model.add(Bidirectional(LSTM(units=64)))
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='tanh'))
    model.add(Dense(1))
    model.compile(optimizer=Adam(learning_rate=1e-3), loss='mse')
    return model

# Inicializar EarlyStopping
early_stopping = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)

# Diccionario para realizar un seguimiento de los productos ya procesados
productos_procesados = {}

# Entrenar y predecir para cada cluster
for cluster in range(num_clusters):
    productos_cluster = sell_in_completo[sell_in_completo["cluster"] == cluster]["product_id"].unique()
    aux_combined = []
    
    for producto in productos_cluster:
        aux = sell_in_completo[sell_in_completo["product_id"] == producto].drop(columns=["product_id", "cluster", "periodo"]).values
        aux_normalizado = scaler.fit_transform(aux)
        aux_combined.append(aux_normalizado)

    if aux_combined:  # Verificar si hay datos para combinar
        aux_combined = np.concatenate(aux_combined, axis=0)
        X, y = crear_secuencias(aux_combined, n_steps, step_ahead=2)
        X = X.reshape((X.shape[0], X.shape[1], n_features))

        # Construir el modelo
        model = build_model()

        # Entrenar el modelo con todos los datos del cluster
        model.fit(X, y, epochs=200, batch_size=32, callbacks=[early_stopping], verbose=0)

        # Realizar predicciones individuales para cada producto del cluster
        for producto in productos_cluster:
            if producto not in productos_procesados:
                try:
                    aux = sell_in_completo[sell_in_completo["product_id"] == producto].drop(columns=["product_id", "cluster", "periodo"]).values
                    aux_normalizado = scaler.fit_transform(aux)
                    ultima_secuencia = aux_normalizado[-n_steps:].reshape((1, n_steps, n_features))
                    prediccion_normalizada = model.predict(ultima_secuencia)
                    prediccion = scaler.inverse_transform(prediccion_normalizada)
                    resultados_por_producto.append({'product_id': producto, 'prediccion_1': prediccion[0][0]})
                    productos_procesados[producto] = True  # Marcar producto como procesado
                except Exception as e:
                    print(f"Error al procesar el producto {producto}: {str(e)}")
                    resultados_por_producto.append({'product_id': producto, 'prediccion_1': 0})
                    productos_procesados[producto] = True  # Marcar producto como procesado

# Convertir la lista de resultados en un DataFrame
predicciones_por_producto = pd.DataFrame(resultados_por_producto)

# Mostrar el DataFrame con las predicciones por producto
print(predicciones_por_producto)

# Guardar el DataFrame con las predicciones por producto en un archivo CSV
predicciones_por_producto.to_csv("C:/Users/vgarciario/desktop/UA MASTER/labo3/lstmclus3.csv", index=False)


c:\Users\vgARCIARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 568ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━

c:\Users\vgARCIARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 580ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Error al procesar el producto 21153: cannot reshape array of size 11 into shape (1,12,1)
Error al procesar el producto 21159: cannot reshape array of size 11 into shape (1,12,1)
Error al procesar el producto 21168: cannot reshape array of 

c:\Users\vgARCIARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 738ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━

c:\Users\vgARCIARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 565ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━

c:\Users\vgARCIARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 552ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Dropout, Activation
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping

# Lista para acumular las filas de resultados
resultados_por_producto = []

# Elegir un número de pasos de tiempo
n_steps = 18  # Ventana de tiempo de 18 meses
n_features = 1  # Cambia esto si tienes más características

# Inicializar scaler
scaler = MinMaxScaler()

# Preparar datos para clustering usando únicamente la característica 'tn'
sell_in_completo['periodo'] = pd.to_datetime(sell_in_completo['periodo'])
sell_in_completo = sell_in_completo.sort_values(by='periodo')

# Crear un DataFrame pivotado
tn_data = sell_in_completo.pivot(index='periodo', columns='id_product', values='tn').fillna(0)

# Normalizar los datos
tn_data_normalizada = scaler.fit_transform(tn_data)

# Aplicar K-means clustering
num_clusters = 6  # Número de clusters
kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(tn_data_normalizada.T)
sell_in_completo['cluster'] = sell_in_completo['id_product'].map(
    dict(zip(tn_data.columns, kmeans.labels_))
)

# Función para crear secuencias de tiempo
def crear_secuencias(datos, n_steps, step_ahead=1):
    X, y = [], []
    for i in range(len(datos)):
        end_ix = i + n_steps
        out_end_ix = end_ix + step_ahead - 1
        if out_end_ix > len(datos) - 1:
            break
        seq_x, seq_y = datos[i:end_ix], datos[out_end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

# Definir la función para construir el modelo
def build_model():
    model = Sequential()
    model.add(Bidirectional(LSTM(units=256, return_sequences=True), input_shape=(n_steps, n_features)))
    model.add(Dropout(0.3))
    model.add(Bidirectional(LSTM(units=128)))
    model.add(Dropout(0.3))
    model.add(Dense(64))
    model.add(Activation('tanh'))
    model.add(Dense(1))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss='mse')
    return model

# Inicializar EarlyStopping
early_stopping = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)

# Diccionario para realizar un seguimiento de los productos ya procesados
productos_procesados = {}

# Entrenar y predecir para cada cluster
for cluster in range(num_clusters):
    productos_cluster = sell_in_completo[sell_in_completo["cluster"] == cluster]["id_product"].unique()
    aux_combined = []
    
    for producto in productos_cluster:
        aux = sell_in_completo[sell_in_completo["id_product"] == producto].drop(columns=["id_product", "cluster", "periodo"]).values
        
        # Rellenar con ceros si el número de datos es menor que n_steps
        if aux.shape[0] < n_steps:
            aux = np.pad(aux, (n_steps - aux.shape[0], 0), 'constant')
        
        aux_normalizado = scaler.fit_transform(aux)
        aux_combined.append(aux_normalizado)

    aux_combined = np.concatenate(aux_combined, axis=0)
    X, y = crear_secuencias(aux_combined, n_steps, step_ahead=2)
    X = X.reshape((X.shape[0], X.shape[1], n_features))

    # Construir el modelo
    model = build_model()

    # Entrenar el modelo con todos los datos del cluster
    model.fit(X, y, epochs=200, batch_size=32, callbacks=[early_stopping], verbose=0)

    # Realizar predicciones individuales para cada producto del cluster
    for producto in productos_cluster:
        if producto not in productos_procesados:
            try:
                aux = sell_in_completo[sell_in_completo["id_product"] == producto].drop(columns=["id_product", "cluster", "periodo"]).values
                
                # Rellenar con ceros si el número de datos es menor que n_steps
                if aux.shape[0] < n_steps:
                    aux = np.pad(aux, (n_steps - aux.shape[0], 0), 'constant')
                    
                aux_normalizado = scaler.fit_transform(aux)
                ultima_secuencia = aux_normalizado[-n_steps:].reshape((1, n_steps, n_features))
                prediccion_normalizada = model.predict(ultima_secuencia)
                prediccion = scaler.inverse_transform(prediccion_normalizada)
                resultados_por_producto.append({'id_product': producto, 'prediccion_1': prediccion[0][0]})
                productos_procesados[producto] = True  # Marcar producto como procesado
            except Exception as e:
                print(f"Error al procesar el producto {producto}: {str(e)}")
                resultados_por_producto.append({'id_product': producto, 'prediccion_1': 0})
                productos_procesados[producto] = True  # Marcar producto como procesado

# Convertir la lista de resultados en un DataFrame
predicciones_por_producto = pd.DataFrame(resultados_por_producto)

# Mostrar el DataFrame con las predicciones por producto
print(predicciones_por_producto)

# Guardar el DataFrame con las predicciones por producto en un archivo CSV
predicciones_por_producto.to_csv("C:/Users/vgarciario/desktop/UA MASTER/labo3/lstmclus126.csv", index=False)


In [11]:
pip install tslearn


     -------------------------------------- 374.4/374.4 kB 1.4 MB/s eta 0:00:00
     ---------------------------------------- 2.7/2.7 MB 4.4 MB/s eta 0:00:00
     ---------------------------------------- 28.1/28.1 MB 4.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [33]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tslearn.clustering import TimeSeriesKMeans
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Dropout, Activation
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping

# Lista para acumular las filas de resultados
resultados_por_producto = []

# Elegir un número de pasos de tiempo
n_steps = 12  # Ventana de tiempo de 12 meses
n_features = 1  # Cambia esto si tienes más características

# Inicializar scaler
scaler = MinMaxScaler()


# Preparar datos para clustering usando únicamente la característica 'tn'
sell_in_completo['periodo'] = pd.to_datetime(sell_in_completo['periodo'])
sell_in_completo = sell_in_completo.sort_values(by='periodo')

# Crear un DataFrame pivotado
tn_data = sell_in_completo.pivot(index='periodo', columns='product_id', values='tn').fillna(0)

# Normalizar los datos
tn_data_normalizada = scaler.fit_transform(tn_data)

# Transponer los datos para que estén en el formato (n_samples, n_timestamps)
tn_data_normalizada = tn_data_normalizada.T


# Aplicar TimeSeriesKMeans con DTW para clustering
num_clusters = 6  # Número de clusters
model_dtw = TimeSeriesKMeans(n_clusters=num_clusters, metric="dtw", random_state=0)
cluster_labels = model_dtw.fit_predict(tn_data_normalizada)

# Asignar cada producto a su cluster
product_clusters = pd.DataFrame({'product_id': tn_data.columns, 'cluster': cluster_labels})

# Unir resultados al dataset original
sell_in_completo.reset_index(inplace=True)
sell_in_completo = sell_in_completo.merge(product_clusters, on='product_id').set_index('periodo')

# Función para crear secuencias de tiempo
def crear_secuencias(datos, n_steps, step_ahead=1):
    X, y = [], []
    for i in range(len(datos)):
        end_ix = i + n_steps
        out_end_ix = end_ix + step_ahead - 1
        if out_end_ix > len(datos) - 1:
            break
        seq_x, seq_y = datos[i:end_ix], datos[out_end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

# Definir la función para construir el modelo
def build_model():
    model = Sequential()
    model.add(Bidirectional(LSTM(units=256, return_sequences=True), input_shape=(n_steps, n_features)))
    model.add(Dropout(0.3))
    model.add(Bidirectional(LSTM(units=128)))
    model.add(Dropout(0.3))
    model.add(Dense(64))
    model.add(Activation('tanh'))
    model.add(Dense(1))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss='mse')
    return model

# Inicializar EarlyStopping
early_stopping = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)

# Diccionario para realizar un seguimiento de los productos ya procesados
productos_procesados = {}

# Entrenar y predecir para cada cluster
for cluster in range(num_clusters):
    productos_cluster = sell_in_completo[sell_in_completo["cluster"] == cluster]["product_id"].unique()
    aux_combined = []
    
    for producto in productos_cluster:
        aux = sell_in_completo[sell_in_completo["product_id"] == producto].drop(columns=["product_id", "cluster", "periodo"]).values
        
        # Rellenar con ceros si el número de datos es menor que n_steps
        if aux.shape[0] < n_steps:
            aux = np.pad(aux, (n_steps - aux.shape[0], 0), 'constant')
        
        aux_normalizado = scaler.fit_transform(aux)
        aux_combined.append(aux_normalizado)

    aux_combined = np.concatenate(aux_combined, axis=0)
    X, y = crear_secuencias(aux_combined, n_steps, step_ahead=2)
    X = X.reshape((X.shape[0], X.shape[1], n_features))

    # Construir el modelo
    model = build_model()

    # Entrenar el modelo con todos los datos del cluster
    model.fit(X, y, epochs=200, batch_size=32, callbacks=[early_stopping], verbose=0)

    # Realizar predicciones individuales para cada producto del cluster
    for producto in productos_cluster:
        if producto not in productos_procesados:
            try:
                aux = sell_in_completo[sell_in_completo["product_id"] == producto].drop(columns=["product_id", "cluster", "periodo"]).values
                
                # Rellenar con ceros si el número de datos es menor que n_steps
                if aux.shape[0] < n_steps:
                    aux = np.pad(aux, (n_steps - aux.shape[0], 0), 'constant')
                    
                aux_normalizado = scaler.fit_transform(aux)
                ultima_secuencia = aux_normalizado[-n_steps:].reshape((1, n_steps, n_features))
                prediccion_normalizada = model.predict(ultima_secuencia)
                prediccion = scaler.inverse_transform(prediccion_normalizada)
                resultados_por_producto.append({'product_id': producto, 'prediccion_1': prediccion[0][0]})
                productos_procesados[producto] = True  # Marcar producto como procesado
            except Exception as e:
                print(f"Error al procesar el producto {producto}: {str(e)}")
                resultados_por_producto.append({'product_id': producto, 'prediccion_1': 0})
                productos_procesados[producto] = True  # Marcar producto como procesado

# Convertir la lista de resultados en un DataFrame
predicciones_por_producto = pd.DataFrame(resultados_por_producto)

# Mostrar el DataFrame con las predicciones por producto
print(predicciones_por_producto)

# Guardar el DataFrame con las predicciones por producto en un archivo CSV
predicciones_por_producto.to_csv("C:/Users/vgarciario/desktop/UA MASTER/labo3/lstmclusdtw.csv", index=False)


KeyError: 'periodo'